In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle

import sys
sys.path.append("/home/z5297792/UNSW-MRes/MRes/modules") 
from utils import dopioe, find_directional_radii, psi_params, fit_psi_params

df_eddies = pd.read_pickle(f'/srv/scratch/z5297792/Chapter2/SEACOFS_26yr_Eddy_Dataset/Eddy_Data/df_eddies_processed_1462_10650.pkl')
df_eddies


,Eddy,Day,Cyc,Lon,Lat,ic,jc,xc,yc,w,q11,q12,q22,Rc,psi0,A,R,Age,Date,fname
0,1,1462,CE,156.969773,-28.461100,179,274,505.897600,1353.012251,-0.000033,1.099922,-0.193920,0.943344,104.677927,71.391686,-0.006515,69.250036,57,1994-01-02,/srv/scratch/z3533156/26year_BRAN2020/outer_av...
1,1,1463,CE,156.993096,-28.433992,179,275,507.149717,1356.633687,-0.000030,1.066764,-0.119714,0.950849,110.542281,77.351002,-0.006330,62.000626,57,1994-01-03,/srv/scratch/z3533156/26year_BRAN2020/outer_av...
2,1,1464,CE,157.097940,-28.381485,181,276,515.130000,1365.650728,-0.000028,1.016800,-0.154298,1.006892,115.245476,75.250400,-0.005666,67.000485,57,1994-01-04,/srv/scratch/z3533156/26year_BRAN2020/outer_av...
3,1,1465,CE,157.094334,-28.315658,181,278,512.570772,1372.426342,-0.000033,0.896874,-0.276325,1.200119,114.738549,72.789618,-0.005529,66.501582,57,1994-01-05,/srv/scratch/z3533156/26year_BRAN2020/outer_av...
4,1,1466,CE,157.106324,-28.346479,181,277,514.728351,1369.599726,-0.000029,0.847358,-0.169788,1.214160,100.425496,58.705551,-0.005821,56.251116,57,1994-01-06,/srv/scratch/z3533156/26year_BRAN2020/outer_av...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105233,2631,10646,AE,157.825620,-37.030656,248,97,850.502045,476.447416,0.000013,0.898473,0.197470,1.156400,114.894173,-41.491437,0.003143,72.751791,24,2019-02-24,/srv/scratch/z3533156/26year_BRAN2020/outer_av...
105234,2631,10647,AE,157.880796,-36.972725,249,98,853.608024,484.316677,0.000014,0.884289,0.308226,1.238286,111.853372,-42.195517,0.003373,73.000933,24,2019-02-25,/srv/scratch/z3533156/26year_BRAN2020/outer_av...
105235,2631,10648,AE,157.868516,-36.874363,248,100,849.759258,494.400614,0.000014,0.883521,0.168561,1.163994,107.180060,-43.102690,0.003752,75.750780,24,2019-02-26,/srv/scratch/z3533156/26year_BRAN2020/outer_av...
105236,2631,10649,AE,157.753058,-36.882303,246,99,840.037603,489.981707,0.000013,0.873263,0.225002,1.203104,100.368133,-37.967048,0.003769,75.750006,24,2019-02-27,/srv/scratch/z3533156/26year_BRAN2020/outer_av...


In [2]:
import netCDF4 as nc
from scipy.interpolate import griddata
import netCDF4 as nc
from scipy.interpolate import RegularGridInterpolator
import time

# Field Data
fname = f'/srv/scratch/z3533156/26year_BRAN2020/outer_avg_01461.nc'
dataset = nc.Dataset(fname)
lon_rho = np.transpose(dataset.variables['lon_rho'], axes=(1, 0))
lat_rho = np.transpose(dataset.variables['lat_rho'], axes=(1, 0))
mask_rho = np.transpose(dataset.variables['mask_rho'], axes=(1, 0))
h =  np.transpose(dataset.variables['h'], axes=(1, 0))
angle = dataset.variables['angle'][0, 0]
z_r = np.load('/srv/scratch/z5297792/z_r.npy')
z_r = np.transpose(z_r, (1, 2, 0))[150, 150, :]
def distance(lat1, lon1, lat2, lon2):
    EARTH_RADIUS = 6357
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat, dlon = lat2 - lat1, lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    return EARTH_RADIUS * 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
j_mid, i_mid = lon_rho.shape[1] // 2, lon_rho.shape[0] // 2
dx = distance(lat_rho[:-1, j_mid], lon_rho[:-1, j_mid],
              lat_rho[1:, j_mid], lon_rho[1:, j_mid])
dy = distance(lat_rho[i_mid, :-1], lon_rho[i_mid, :-1],
              lat_rho[i_mid, 1:], lon_rho[i_mid, 1:])
x_grid = np.insert(np.cumsum(dx), 0, 0)
y_grid = np.insert(np.cumsum(dy), 0, 0)
X_grid, Y_grid = np.meshgrid(x_grid, y_grid, indexing='ij')


In [3]:
def compute_vert_data(df_eddies, eddy, X, Y, z_r, r=30):

    df = df_eddies[df_eddies.Eddy == eddy].copy()
    dic_data = {f'Eddy{eddy}': {}}
    fnumber_old = None
    x, y = X[:,0], Y[0,:]
    timer = 0

    # DOPIOE wont work if too close to boundary
    dx = np.max(np.diff(x))  # spacing in x-direction
    dy = np.max(np.diff(y))  # spacing in y-direction
    cell_size = np.max([dx, dy])        # max cell size in Euclidean units
    margin = int(np.ceil(r / cell_size))    

    for t, data in df.iterrows():
        day, fname, xc_surf, yc_surf, w_surf = data['Day'], data['fname'], data['xc'], data['yc'], data['w']
        fnumber = fname[-8:-3]

        # Only load if file changes
        if fnumber != fnumber_old:
            u_raw = np.load(f'/srv/scratch/z5297792/Climatology/u_v/u_{fnumber}.npy')
            v_raw = np.load(f'/srv/scratch/z5297792/Climatology/u_v/v_{fnumber}.npy')
            u_raw = np.where(np.abs(u_raw) > 1e30, np.nan, u_raw).astype(float)
            v_raw = np.where(np.abs(v_raw) > 1e30, np.nan, v_raw).astype(float)
            with nc.Dataset(fname) as dataset:
                ocean_time = dataset.variables['ocean_time'][:].data / 86400
            fnumber_old = fnumber
        t_rel = np.where(ocean_time == day)[0][0]

        u_t, v_t = u_raw[:, :, :, t_rel], v_raw[:, :, :, t_rel]

        df_data = []
        xc, yc = xc_surf, yc_surf
        xc_old, yc_old = xc_surf, yc_surf

        # for k in range(24):
        for k in range(u_t.shape[-1]):
            u, v = u_t[:, :, k], v_t[:, :, k]

            R_grid = np.hypot(xc - X, yc - Y)
            ic, jc = map(int, np.unravel_index(np.argmin(R_grid), R_grid.shape))
            
            if (ic < margin or ic >= X.shape[0] - margin or
                jc < margin or jc >= X.shape[1] - margin):
                break

            # horizontal transect (constant y = y[jc])
            x_mask = np.abs(x - xc) <= r
            x1 = x[x_mask]
            y1 = np.full_like(x1, y[jc])
            u1 = u[x_mask, jc]
            v1 = v[x_mask, jc]
        
            # vertical transect (constant x = x[ic])
            y_mask = np.abs(y - yc) <= r
            y2 = y[y_mask]
            x2 = np.full_like(y2, x[ic])
            u2 = u[ic, y_mask]
            v2 = v[ic, y_mask]

            if np.any(np.isnan(u1)) or np.any(np.isnan(u2)):
                break

            xc, yc, w, Q, _, _, A0 = dopioe(x1, y1, u1, v1, x2, y2, u2, v2)

            if (np.sign(w) != np.sign(w_surf)) or (np.hypot(xc_old - xc, yc_old - yc) > 50):
                break
            
            xc_old, yc_old = xc, yc

            w *= 1e-3 # to s^-1

            radii = find_directional_radii(u, v, X, Y, xc, yc, Q)
            R = np.mean([radii['up'], radii['right'], radii['down'], radii['left']])

            q11, q12, q22 = Q[0,0], Q[0,1], Q[1,1]
            dx, dy = X - xc, Y - yc
            rho2 = q11*dx**2 + 2*q12*dx*dy + q22*dy**2
            rho_search = np.sqrt(np.where(rho2 < 0, np.nan, rho2))
            
            mask_outer = rho_search < max(min(R*1.75, 200), 30)
            axi, ayi, aui, avi = X[mask_outer], Y[mask_outer], u[mask_outer], v[mask_outer]
            
            df = psi_params(xc, yc, Q, axi, ayi, aui, avi)
            Rc, psi0, A = fit_psi_params(df.rho2, df.Qr, df.vt, A0=A0, Rc_max=300)

            if np.sign(A) != np.sign(w):
                Rc, psi0, A = np.nan, np.nan, A0

            df_data.append({
                'x': xc, 'y': yc, 'Q': Q, 'w': w,
                'Rc': Rc, 'psi0': psi0, 'A': A, 'R': R,
                'z': k, 'Depth': z_r[k]
            })

        if df_data:
            df_data = pd.DataFrame(df_data)
            dx = df_data['x'].diff()
            dy = df_data['y'].diff()
            df_data['TD'] = np.hypot(df_data['x'] - df_data.iloc[0]['x'],
                                  df_data['y'] - df_data.iloc[0]['y'])
            dic_data[f'Eddy{eddy}'][f'Day{day}'] = df_data
        else:
            dic_data[f'Eddy{eddy}'][f'Day{day}'] = pd.DataFrame(columns=['x', 'y', 'Q', 'w', 'Rc', 'psi0', 'A', 'R', 'z','Depth'])

        # if timer % 10 == 0:
        #     print(f'{timer / len(df) * 100:.1f}%')
        #     timer += 1
        
    return dic_data
    

In [4]:
dic_vert_eddies = {}
for e, eddy in enumerate(df_eddies.Eddy.unique()):
    if eddy >= 2039:
        dic = compute_vert_data(df_eddies, eddy, X_grid, Y_grid, z_r)
        dic_vert_eddies = dic_vert_eddies | dic 
    
        with open(f"/srv/scratch/z5297792/Chapter2/SEACOFS_26yr_Eddy_Dataset/Eddy_Data/Vertical_Eddy_Dataset/dic_vert_eddies_{2039}_{df_eddies.Eddy.max()}_no_processing.pkl", "wb") as f:
            pickle.dump(dic_vert_eddies, f)
    
        if e % 10 == 0:
            print(f'{e+1}/{len(df_eddies.Eddy.unique())}')



/home/z5297792/UNSW-MRes/MRes/modules/utils.py:456: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(vt_model, rho2, vt, p0=[A0, Rc0],
/home/z5297792/UNSW-MRes/MRes/modules/utils.py:456: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(vt_model, rho2, vt, p0=[A0, Rc0],


2041/2631


/home/z5297792/UNSW-MRes/MRes/modules/utils.py:456: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(vt_model, rho2, vt, p0=[A0, Rc0],
/home/z5297792/UNSW-MRes/MRes/modules/utils.py:456: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(vt_model, rho2, vt, p0=[A0, Rc0],
/home/z5297792/.local/lib/python3.10/site-packages/scipy/optimize/_lsq/common.py:115: RuntimeWarning: divide by zero encountered in divide
  phi_prime = -np.sum(suf ** 2 / denom**3) / p_norm
/home/z5297792/UNSW-MRes/MRes/modules/utils.py:456: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(vt_model, rho2, vt, p0=[A0, Rc0],
/home/z5297792/UNSW-MRes/MRes/modules/utils.py:456: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(vt_model, rho2, vt, p0=[A0, Rc0],
/home/z5297792/UNSW-MRes/MRes/modules/utils.py:456: OptimizeWarning: Covariance of the parameters cou

2051/2631


/home/z5297792/UNSW-MRes/MRes/modules/utils.py:456: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(vt_model, rho2, vt, p0=[A0, Rc0],
/home/z5297792/UNSW-MRes/MRes/modules/utils.py:456: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(vt_model, rho2, vt, p0=[A0, Rc0],
/home/z5297792/UNSW-MRes/MRes/modules/utils.py:456: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(vt_model, rho2, vt, p0=[A0, Rc0],
/home/z5297792/UNSW-MRes/MRes/modules/utils.py:456: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(vt_model, rho2, vt, p0=[A0, Rc0],
/home/z5297792/UNSW-MRes/MRes/modules/utils.py:456: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(vt_model, rho2, vt, p0=[A0, Rc0],
/home/z5297792/UNSW-MRes/MRes/modules/utils.py:456: OptimizeWarning: Covariance of the parameters could not be esti

...

2611/2631


/home/z5297792/UNSW-MRes/MRes/modules/utils.py:456: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(vt_model, rho2, vt, p0=[A0, Rc0],
/home/z5297792/UNSW-MRes/MRes/modules/utils.py:456: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(vt_model, rho2, vt, p0=[A0, Rc0],
/home/z5297792/UNSW-MRes/MRes/modules/utils.py:456: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(vt_model, rho2, vt, p0=[A0, Rc0],
/home/z5297792/UNSW-MRes/MRes/modules/utils.py:456: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(vt_model, rho2, vt, p0=[A0, Rc0],
/home/z5297792/UNSW-MRes/MRes/modules/utils.py:456: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(vt_model, rho2, vt, p0=[A0, Rc0],
/home/z5297792/UNSW-MRes/MRes/modules/utils.py:456: OptimizeWarning: Covariance of the parameters could not be esti

2621/2631


/home/z5297792/UNSW-MRes/MRes/modules/utils.py:456: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(vt_model, rho2, vt, p0=[A0, Rc0],
/home/z5297792/UNSW-MRes/MRes/modules/utils.py:456: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(vt_model, rho2, vt, p0=[A0, Rc0],
/home/z5297792/UNSW-MRes/MRes/modules/utils.py:456: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(vt_model, rho2, vt, p0=[A0, Rc0],
/home/z5297792/UNSW-MRes/MRes/modules/utils.py:456: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(vt_model, rho2, vt, p0=[A0, Rc0],
/home/z5297792/UNSW-MRes/MRes/modules/utils.py:456: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(vt_model, rho2, vt, p0=[A0, Rc0],
/home/z5297792/UNSW-MRes/MRes/modules/utils.py:456: OptimizeWarning: Covariance of the parameters could not be esti

2631/2631
